## 다중 시퀀스 처리

Tansformers API를 사용하여 아래의 의문을 해결해보자

1. 다중 시퀀스를 어떻게 처리할까?
2. 길이가 다른 여러 개의 시퀀스를 어떻게 처리할까?
3. 어휘집의 인덱스들만 입력하면 모델이 잘 동작할까?
4. 길이가 너무 긴 시퀀스는 어떻게 처리할까?

### 모델은 배치 형태의 입력을 요구한다

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)

ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)

model(input_ids)

RuntimeError: The size of tensor a (14) must match the size of tensor b (512) at non-singleton dimension 1

위 문제는 우리가 모델에 하나의 단일 시퀀스를 입력해서 발생한다.

Transformers 모델은 기본적으로 __다중 문장 시퀀스를 한번에 입력하기를 요구한다.__

In [2]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


In [4]:
# 오류 발생 코드에서 input_ids에 새로운 차원 추가

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


Batching: 모델을 통해 한번에 여러 문장을 입력하는 동작\
문장이 하나만 있는 경우 아래와 같이 단일 시퀀스로 배치를 빌드할 수 있다

In [5]:
batch_ids = [ids, ids]

배치 처리를 통해서 모델이 여러 문장을 동시에 입력받을 수 있도록 한다

* 다음 문제: 각 문장 길이가 다른 경우

padding: 문제 해결을 위해 입력을 채운다

### 입력을 패딩

직사각형 리스트는 텐서로 변환할 수 없다\
패딩을 사용하여 텐서를 직사각형 모양으로 만든다\
-> 패딩: 길이가 더 짧은 문장에 패딩 토큰이라는 특수 단어를 추가하여 모든 문장이 동일한 길이를 갖도록 함

예)

```python
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]
```

패딩 토큰의 식별자(ID)는 `tokenizer.pad_token_id` 에 지정되어 있음

In [7]:
# 두 개의 시퀀스를 모델에 입력하기
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


__문제__

트랜스포머 모델이 패딩 토큰 또한 집중하기 때문에 패딩 토큰도 고려한다

sol) <span style="background-color:blue">어텐션 레이어가 패딩 토큰을 무시하도록 지시: `attention_mask` 사용</sapn>

### 어텐션 마스크

* 입력 식별자 텐서와 형태가 정확하게 동일한 텐서
* 0과 1로 채워짐
    * 1: 해당 토큰에 주의를 기울여야 함
    * 0: 해당 토큰을 무시해야 함

In [8]:
# attention mask 를 이용해 예제 완성
batch_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batch_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


### 길이가 더 긴 시퀀스

모델을 사용할 때, 모델에 입력할 수 있는 시퀀스의 길이에 제한이 있고\
최대 토큰 시퀀스보다 더 긴 시퀀스를 처리하라고 하면 오류가 발생한다

sol)

1. 길이가 더 긴 시퀀스를 지원하는 모델 사용
2. truncation

In [9]:
# truncation

max_sequence_length = 512
sequence = sequence[:max_sequence_length]